In [148]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
datas = pd.read_csv("train.csv")
datas.head()
datas.info()
y_col = "Survived"
x_cols = datas.columns.drop(["PassengerId","Survived","Name","Ticket","Fare","Cabin","Embarked"])
print(x_cols)
#!pip install autograd
#!pip install sklearn
from autograd import numpy
from autograd import grad

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch'], dtype='object')


In [149]:
for i in range(len(datas["Sex"])):
    if datas["Sex"][i] == "male":
        datas["Sex"][i] = 0
    else:
        datas["Sex"][i] = 1

mean_of_age=datas['Age'].mean()
isnalst = datas["Age"].isna()
for i in range(len(isnalst)):
    if isnalst[i]:
        datas["Age"][i] = mean_of_age

print(datas.head())

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(datas[x_cols])
X_scaled = numpy.hstack((numpy.ones((X_scaled.shape[0],1)),X_scaled))

y = datas[y_col].values

print(f"x.s={X_scaled.shape} y.s={y.shape}")


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris   0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...   1  38.0      1      0   
2                             Heikkinen, Miss. Laina   1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)   1  35.0      1      0   
4                           Allen, Mr. William Henry   0  35.0      0      0   

             Ticket     Fare Cabin Embarked  
0         A/5 21171   7.2500   NaN        S  
1          PC 17599  71.2833   C85        C  
2  STON/O2. 3101282   7.9250   NaN        S  
3            113803  53.1000  C123        S  
4            373450   8.0500   NaN        S  
x.s=(891, 6) y.s=(891,)


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [150]:
def logistic(z):
    return 1/(1+numpy.exp(-z))
def logistic_model(params,x):
    z = numpy.dot(X_scaled,params)
    y = logistic(z)
    return y
def log_loss(params,model,x,y,_lambda=1.0):
    y_pred = model(params,x)

    return -numpy.mean(y * numpy.log(y_pred) + (1-y) * numpy.log(1 - y_pred)) \
#+ _lambda * numpy.sum(params[1:]**2)

In [151]:
gradient = grad(log_loss)

In [152]:
type(gradient)

function

In [153]:
max_iter = 3000
alpha = 0.01
params = numpy.zeros(X_scaled.shape[1])
descent = numpy.ones(X_scaled.shape[1])
i = 0
print(params)
while (numpy.linalg.norm(descent) > 0.001 and i<max_iter):
    descent = gradient(params,logistic_model,X_scaled,y)
    params = params - descent * alpha
    i += 1
    if i%100 == 0:
        print(f"--{params}--")

[0. 0. 0. 0. 0. 0.]
--[-0.09468297 -0.12768749  0.08991835 -0.04054402 -0.00856585 -0.000856  ]--
--[-1.56716285e-01 -2.28714957e-01  1.85338919e-01 -6.93418158e-02
 -1.50454933e-02  1.46325427e-05]--
--[-0.19787396 -0.31177371  0.28219864 -0.09065067 -0.02023728  0.00185386]--
--[-0.22563405 -0.38243907  0.37804639 -0.10717258 -0.02464472  0.00418953]--
--[-0.24473625 -0.44429468  0.47143687 -0.12061804 -0.02858416  0.00672979]--
--[-0.25819686 -0.49967234  0.56155378 -0.13207349 -0.03225452  0.0092957 ]--
--[-0.26794426 -0.55011421  0.64797641 -0.14223139 -0.03578065  0.01177947]--
--[-0.27521601 -0.59665992  0.73053275 -0.15153419 -0.0392405   0.01411828]--
--[-0.28080868 -0.64002588  0.80920556 -0.16026485 -0.04268216  0.01627774]--
--[-0.28523622 -0.68071838  0.88407185 -0.16860436 -0.04613469  0.01824141]--
--[-0.28883112 -0.7191055   0.95526384 -0.17666838 -0.04961504  0.02000401]--
--[-0.29180912 -0.75546324  1.0229438  -0.18453072 -0.05313248  0.0215671 ]--
--[-0.29431089 -0.7

In [154]:
def decision_boundary(y):
    return 1 if y>=0.5 else 0

In [155]:
decision_boundary = np.vectorize(decision_boundary)

In [156]:
def classify(predictions):
    return decision_boundary(predictions).flatten()

tdatas=pd.read_csv("test.csv")
xt_cols = tdatas.columns.drop(["PassengerId","Name","Ticket","Fare","Cabin","Embarked"])

for i in range(len(tdatas["Sex"])):
    if tdatas["Sex"][i] == "male":
        tdatas["Sex"][i] = 0
    else:
        tdatas["Sex"][i] = 1

mean_of_age=tdatas['Age'].mean()
isnalst = tdatas["Age"].isna()
for i in range(len(isnalst)):
    if isnalst[i]:
        tdatas["Age"][i] = mean_of_age

Xt_scaled = min_max_scaler.fit_transform(tdatas[xt_cols])
Xt_scaled = numpy.hstack((numpy.ones((Xt_scaled.shape[0],1)),Xt_scaled))
dec = classify(logistic_model(params,Xt_scaled))
print(dec)

c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


[0 1 1 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 0 0
 0 1 1 1 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0
 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0
 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1
 0 1 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 1
 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0
 0 1 0 1 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 1 1 1
 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 1 0 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1
 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0
 0 0 1 0 1 1 0 0 0 0 0 0 

c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [160]:
odatas=pd.read_csv("gender_submission.csv")

for i in range(len(odatas["Survived"])):
    odatas["Survived"][i] = dec[i]
print(odatas)
odatas.to_csv("./result.csv",index=False)

     PassengerId  Survived
0            892         0
1            893         1
2            894         1
3            895         1
4            896         0
..           ...       ...
413         1305         0
414         1306         0
415         1307         1
416         1308         1
417         1309         1

[418 rows x 2 columns]
